In [1]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torch.utils.data import DataLoader, Dataset
import pandas as pd

import torchvision
import numpy as np
import torchvision.transforms as transforms

import os
import argparse

import backbones.cifar as models
from datasets import MNIST
from Utils import adjust_learning_rate, progress_bar, Logger, mkdir_p, Evaluation
from openmax import compute_train_score_and_mavs_and_dists,fit_weibull,openmax
from Modelbuilder import Network
from Plotter import plot_feature

In [2]:
model_names = sorted(name for name in models.__dict__
    if not name.startswith("__")
    and callable(models.__dict__[name]))
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [3]:
#准备数据集
class MyDataset(Dataset):
   # TensorDataset继承Dataset, 重载了__init__, __getitem__, __len__
   # 能够通过index得到数据集的数据，能够通过len得到数据集大小

   def __init__(self, data_tensor, target_tensor):
       self.data_tensor = data_tensor
       self.target_tensor = target_tensor

   def __getitem__(self, index):
       return self.data_tensor[index], self.target_tensor[index]

   def __len__(self):
       return self.data_tensor.size(0)


设置参数

In [4]:
arch = 'MyNet'
train_class_num = 2
embed_dim = 2
resume = ''
lr = 0.00001
evaluate = False
es = 1


In [5]:
# checkpoint
checkpoint = 'checkpoints_ipynb/mnist/' + arch
if not os.path.isdir(checkpoint):
    mkdir_p(checkpoint)

# folder to save figures
plotfolder = 'checkpoints_ipynb/mnist/' + arch + '/plotter'
if not os.path.isdir(plotfolder):
    mkdir_p(plotfolder)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


cpu


数据准备

In [7]:
print('==> Preparing data..')
train_url = "E:\exercise_4\Kmeans/t_n_fequence_feature0320/n_fequence_feature_ -label.csv" 
names = ['feature0','feature1','feature2','feature3','feature4','feature5',
'feature6','feature7','feature8','feature9','feature10','feature11',
'feature12','feature13','feature14','feature15',
'feature16','feature17','class'] 
train_dataset = pd.read_csv(train_url, names=names)
train_array = train_dataset.values
x_train = train_array[:,0:18]
y_train = train_array[:,18]

test_url = "E:\exercise_4\Kmeans/t_n_fequence_feature0320/s_n_fequence_feature_-label.csv" 
names = ['feature0','feature1','feature2','feature3','feature4','feature5',
'feature6','feature7','feature8','feature9','feature10','feature11',
'feature12','feature13','feature14','feature15',
'feature16','feature17','class'] 
test_dataset = pd.read_csv(test_url, names=names)
test_array = test_dataset.values
x_test = test_array[:,0:18]
y_test = test_array[:,18]

==> Preparing data..


In [8]:
# 生成数据
# 训练数据
x_train_all = np.array(x_train).astype(np.float32)
train_dataset_X = torch.from_numpy(x_train_all)
train_dataset_Y = np.array(y_train).astype(np.int64)

trainset = MyDataset(train_dataset_X, train_dataset_Y)  # 将数据封装成Dataset
print('tensor_data[0]: ', trainset[0])
print('len(tensor_data): ', len(trainset))

# 测试数据
x_test_all = np.array(x_test).astype(np.float32)
test_dataset_X = torch.from_numpy(x_test_all)
test_dataset_Y = torch.tensor(y_test.astype(float)).long()

testset = MyDataset(test_dataset_X, test_dataset_Y)  # 将数据封装成Dataset
print('tensor_data[0]: ', testset[0])
print('len(tensor_data): ', len(testset))

tensor_data[0]:  (tensor([-1.2611e-03, -5.0107e-03, -5.5262e-04, -4.3407e-03, -5.1200e-05,
        -3.4819e-03, -4.9054e+00,  3.2057e+00,  1.0536e+01,  2.0784e+00,
         1.1622e+01,  3.8205e+00,  1.1497e+02,  3.2958e+01,  7.7813e+01,
         3.1822e+04,  3.2292e+00,  1.3294e+01]), 1)
len(tensor_data):  54
tensor_data[0]:  (tensor([-4.0345e-04, -3.9042e-03,  2.8506e-02,  1.5831e-02,  3.8812e-02,
         3.3401e-02,  2.1563e-01,  3.2291e+00,  1.0930e+01,  7.2528e+00,
         1.0932e+01,  7.9391e+00,  8.8870e+01,  6.6000e+01,  1.3937e+02,
         5.5205e+04,  2.0221e+00,  6.5261e+00]), tensor(2))
len(tensor_data):  36


In [73]:
trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)
testloader = DataLoader(testset, batch_size=1, shuffle=True, num_workers=0)
for data, target in trainloader: 
    ad = data
    ta = target
print(ad)
print(ta)

tensor([[ 1.1665e-03,  6.6921e-03,  3.8906e-02,  3.2917e-02,  5.2492e-02,
          5.5909e-02, -3.0518e-01,  3.6399e+00,  2.8547e+00,  3.5485e+00,
          2.8709e+00,  5.0834e+00,  9.6102e+01,  4.4271e+01,  6.4052e+01,
          2.5683e+04,  3.6889e+00,  1.6853e+01]])
tensor([1])


建立模型

In [74]:
net = Network(backbone=arch, num_classes=train_class_num, embed_dim=embed_dim)
fea_dim = net.classifier.in_features
net = net.to(device)
print(net)

Network(
  (backbone): MyNet(
    (conv1_1): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
    (prelu1_1): PReLU(num_parameters=1)
    (conv1_2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (prelu1_2): PReLU(num_parameters=1)
    (conv2_1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (prelu2_1): PReLU(num_parameters=1)
    (conv2_2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (prelu2_2): PReLU(num_parameters=1)
    (conv3_1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (prelu3_1): PReLU(num_parameters=1)
    (conv3_2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
    (prelu3_2): PReLU(num_parameters=1)
  )
  (embeddingLayer): Sequential(
    (0): Linear(in_features=2304, out_features=2, bias=True)
    (1): PReLU(num_parameters=1)
  )
  (classifier): Linear(in_features=2, out_features=2, bias=True)
)


训练

In [75]:
if resume:
# Load checkpoint.
    if os.path.isfile(resume):
        print('==> Resuming from checkpoint..')
        checkpoint = torch.load(resume)
        net.load_state_dict(checkpoint['net'])
        # best_acc = checkpoint['acc']
        # print("BEST_ACCURACY: "+str(best_acc))
        start_epoch = checkpoint['epoch']
        logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
    else:
        print("=> no checkpoint found at '{}'".format(resume))
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Epoch', 'Learning Rate', 'Train Loss','Train Acc.', 'Test Loss', 'Test Acc.'])

In [76]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

In [77]:
# Training
def train(net,trainloader,optimizer,criterion,device):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.reshape(1,1,1,18)
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        _, outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return train_loss/(batch_idx+1), correct/total

In [78]:
def save_model(net, acc, epoch, path):
    print('Saving..')
    state = {
        'net': net.state_dict(),
        'testacc': acc,
        'epoch': epoch,
    }
    torch.save(state, path)

In [81]:
def test(epoch, net,trainloader,  testloader,criterion, device):
    net.eval()

    test_loss = 0
    correct = 0
    total = 0

    scores, labels = [], []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):

            inputs = inputs.reshape(1,1,1,18)
            
            inputs, targets = inputs.to(device), targets.to(device)
            _, outputs = net(inputs)
            # loss = criterion(outputs, targets)
            # test_loss += loss.item()
            # _, predicted = outputs.max(1)
            scores.append(outputs)
            labels.append(targets)

            # total += targets.size(0)
            # correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader))

    # Get the prdict results.
    scores = torch.cat(scores,dim=0).cpu().numpy()
    labels = torch.cat(labels,dim=0).cpu().numpy()
    scores = np.array(scores)[:, np.newaxis, :]
    labels = np.array(labels)

    # Fit the weibull distribution from training data.
    print("Fittting Weibull distribution...")

    _, mavs, dists = compute_train_score_and_mavs_and_dists(train_class_num, trainloader, device, net)
    



In [82]:
es = 1
epoch=0
plot_max = 0
plot_quality = 200
if not evaluate:
    for epoch in range(start_epoch, es):
        print('\nEpoch: %d   Learning rate: %f' % (epoch+1, optimizer.param_groups[0]['lr']))
        adjust_learning_rate(optimizer, epoch, lr, step=20)
        train_loss, train_acc = train(net,trainloader,optimizer,criterion,device)
        save_model(net, None, epoch, os.path.join(checkpoint,'last_model.pth'))
        test_loss, test_acc = 0, 0
        logger.append([epoch+1, optimizer.param_groups[0]['lr'], train_loss, train_acc, test_loss, test_acc])
        # plot_feature(net, trainloader, device, plotfolder, epoch=epoch,
        #                 plot_class_num=train_class_num, maximum=plot_max, plot_quality=plot_quality)
        test(epoch, net, trainloader, testloader, criterion, device)
        


Epoch: 1   Learning rate: 0.000010
 [===============================================================>.]  Step: 14ms | Tot: 832ms | Loss: 0.818 | Acc: 57.407% (31/54) 54/54 =====================>..........................................]  Step: 30ms | Tot: 346ms | Loss: 0.917 | Acc: 35.000% (7/20) 20/54 ===>....................]  Step: 7ms | Tot: 597ms | Loss: 0.787 | Acc: 44.737% (17/38) 38/54 ==========>..]  Step: 8ms | Tot: 817ms | Loss: 0.833 | Acc: 56.604% (30/53) 53/54 
Saving..
 [===============================================================>.]  Step: 2ms | Tot: 202ms 36/36 .......................................................]  Step: 39ms | Tot: 43ms 3/36 ====================>.....................]  Step: 9ms | Tot: 150ms 25/36 ...............]  Step: 4ms | Tot: 164ms 28/36 
Fittting Weibull distribution...


ValueError: not enough values to unpack (expected 3, got 2)

In [86]:
import libmr

mr = libmr.MR()
tailtofit = [0.43512735, 0.72709793, 0.7634604 , 0.9856832 , 1.38425899]
print(tailtofit)
mr.fit_high(tailtofit, len(tailtofit))
print(mr)

[0.43512735, 0.72709793, 0.7634604, 0.9856832, 1.38425899]
1.552454219243758037905e+00 4.834796487802818454327e+00  2.538549965943251329747e+00 9.494058163839286246244e-01 2.649326762877164043175e+01 8.823093249956448502758e-01  1 5.000000000000000000000e+00 1 5 1 4.351273500000000238330e-01 0



In [87]:
import libmr

mr = libmr.MR()
tailtofit = [1, 2, 3 , 4 , 5]
print(tailtofit)
mr.fit_high(tailtofit, len(tailtofit))
print(mr)

[1, 2, 3, 4, 5]
3.394289398063347729106e+00 2.293801464918516774816e+00  9.485253708554127527464e+00 1.214643368728770322917e+00 1.461669044566115616135e+01 3.599669282196622521752e-01  1 5.000000000000000000000e+00 1 5 1 1.000000000000000000000e+00 0

